<a href="https://colab.research.google.com/github/fjtm/deep-fake-voice-recognition/blob/feature%2Fstart/base_ml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone -b feature/start https://github.com/fjtm/deep-fake-voice-recognition.git
%cd deep-fake-voice-recognition/

/content/deep-fake-voice-recognition


In [2]:
!pip install feature-engine

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd

In [5]:
csv_files_path = f'/content/drive/My Drive/deep-fake-voice-recognition/data/'

In [6]:
train = pd.read_csv(csv_files_path+"train.csv")
test = pd.read_csv(csv_files_path+"test.csv")

In [7]:
target_column = "target"
index_columns = "ind_num"
not_data_columns = ["label", "ind", "target","ind_num"]

In [8]:
X_train, y_train = train.drop(not_data_columns, axis = 1), train[target_column]
X_test, y_test = test.drop(not_data_columns, axis = 1), test[target_column]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((1857240, 58), (1857240,), (716918, 58), (716918,))

In [9]:
y_train.value_counts()

0    1584114
1     273126
Name: target, dtype: int64

In [84]:
from feature_engine.selection import SmartCorrelatedSelection, DropCorrelatedFeatures
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
# class ClfSwitcher(BaseEstimator):
#     def __init__(
#         self,
#         estimator = RandomForestClassifier(),
#     ):
#         """
#         A Custom BaseEstimator that can switch between classifiers.
#         :param estimator: sklearn object - The classifier
#         """

#         self.estimator = estimator


#     def fit(self, X, y=None, **kwargs):
#         self.estimator.fit(X, y)
#         return self


#     def predict(self, X, y=None):
#         return self.estimator.predict(X)


#     def predict_proba(self, X):
#         return self.estimator.predict_proba(X)


#     def score(self, X, y):
#         return self.estimator.score(X, y)

In [108]:
pipe = Pipeline([
    ('feature_selection', SmartCorrelatedSelection()),
    ('clf', RandomForestClassifier())
])

In [117]:
parameters = [
    {
        'feature_selection' : ['passthrough'],
        'clf': [RandomForestClassifier()],
        'clf__n_estimators': [16],
        'clf__max_depth' : [8, 16]
    },
    {
        'feature_selection' : ['passthrough', SmartCorrelatedSelection(), DropCorrelatedFeatures()],
        'clf': [QuadraticDiscriminantAnalysis()]
    },
    {
        'feature_selection' : ['passthrough', SmartCorrelatedSelection(), DropCorrelatedFeatures()],
        'clf': [GaussianNB()],
    },
    {
        'feature_selection' : ['passthrough', SmartCorrelatedSelection(), DropCorrelatedFeatures()],
        'clf': [LogisticRegression()],
        # 'clf__C' : [1e-5, 1e-3, 1e-1, 1, 10]
    },
]


In [118]:
from sklearn.model_selection import GridSearchCV

In [119]:
clf_gs = (
    GridSearchCV(
        estimator=pipe,
        param_grid=parameters,
        n_jobs = -1,
        scoring = 'roc_auc',
        return_train_score = True,
        verbose = 10,
        cv = 3
        )
    )

In [120]:
clf_best_model = clf_gs.fit(X_train, y_train)

Fitting 3 folds for each of 11 candidates, totalling 33 fits


In [124]:
import joblib
model_files_path = f'/content/drive/My Drive/deep-fake-voice-recognition/models/'
# joblib.dump(clf_best_model, f'{model_files_path}baseline_model.pkl')

['/content/drive/My Drive/deep-fake-voice-recognition/models/baseline_model.pkl']

In [126]:
clf_best_model = joblib.load(f'{model_files_path}baseline_model.pkl')

In [131]:
clf_best_model.best_estimator_

Pipeline(steps=[('feature_selection', 'passthrough'),
                ('clf', RandomForestClassifier(max_depth=16, n_estimators=16))])

In [128]:
clf_best_model.score(X_train, y_train)

0.9924605970263496

In [129]:
clf_best_model.score(X_test, y_test)

0.9668828710712866